In [1]:
import numpy as np
import matplotlib.pyplot as plt
import dataprocess as dp
import waveform_spin as wf
import mpmath as mp
import fishermatrix_massratio as fh
import fishermatrix as fm
import pandas as pd
pi = np.pi

In [3]:
logmass = np.log10(1e6)
logmass2 = 1e6
para = [logmass,0.6,1]
rd_ang = [pi/4,pi/4,pi/4]
print(fh.ls_fm([logmass,0.6,1],[pi/4,pi/4,pi/4]))
print(fm.ls_fm([logmass2,2.2883235749605624,1],[pi/4,pi/4,pi/4]))

(0.012355959193758577, 0.0325768390335081, 0.049031886144281424, 0.03794909447542253, 0.046375706302333584, 0.047041874885067565, 0.018841499714484105, 0.0833819166759293)
(0.0025584367901367823, 0.023527190966864186, 0.00036448531674778377, 0.0033216326029808425, 0.01586660155330519, 0.11864531158511457, 0.003312983766765493, 0.04046887164133881)


In [19]:
fh.tq_fm([logmass,0.6,1],[pi/4,pi/4,pi/4])

(0.002900933721401066,
 0.007311461616259999,
 0.011210006149449956,
 0.008640841690987036,
 0.01107312768370292,
 0.010443399268455686,
 0.005318294954478356,
 0.03409624500868881)

In [6]:
freq = np.arange(1e-5,1,1e-5)
print(fh.gp_ls_fm(freq,para,rd_ang,0.1))
print(fm.gp_ls_fm(freq,[logmass2,2.2883235749605624,1],[pi/4,pi/4,pi/4],0.1))

(0.012530623156019376, 0.03302673850489336, 0.04973619811763277, 0.03848368786994554, 0.04702212330478584, 0.04760223667682421, 0.019045504278858403, 0.08376795200506909)
(0.011009997875296247, 0.0991322840271624, 0.0006338795915466419, 0.004374593079933688, 0.0646198116615572, 0.5300632160181589, 0.005698900149148824, 0.052677984313940426)


In [2]:
data = pd.read_fwf('population_data/Klein16_Q3delays.txt', header=None)
print(data.iloc[1,:])
inx = 1

red = data.iloc[inx,0]
m1 = data.iloc[inx,1]
m2 = data.iloc[inx,2]
s1 = data.iloc[inx,3]
s2 = data.iloc[inx,4]
rem_s = data.iloc[inx,9]
rem_mass = np.log10(data.iloc[inx,10])

eta = m1*m2/(m1+m2)**2
para = [rem_mass,eta,red,s1,s2,rem_s]


0             1.3042
1           521650.0
2           235460.0
3             0.3028
4             0.6928
5            0.09315
6            0.11335
7            0.13715
8             5.5425
9            0.74993
10          719150.0
11            158.32
12            174.53
13            32.597
14     68916000000.0
15         2038000.0
16        .26688E+09
17        26521000.0
18        .91137E+01
19           93248.0
20        .65327E-02
21    100000000000.0
22          0.000125
Name: 1, dtype: object


In [13]:
eta = 0.1
j = eta*(2*np.sqrt(3)-3.5171*eta+2.5763*eta**2)
print(j)

0.31381546151377543


In [2]:
j=0.6
eta = 0.45504534927350593+\
    1863.7071217428343 / (9.774183841831808e10 - 8.9603842815e10 * j + 299339.0098450251 *np.sqrt(1.3997382578590552e11 - 1.9548367683663617e11 * j+ 8.9603842815e10 * j**2))**(1/3)\
    -0.00012938451784859424 * (9.774183841831808e10 - 8.9603842815e10 * j + 299339.0098450251 * np.sqrt(1.3997382578590552e11 - 1.9548367683663617e11 * j + 8.9603842815e10 * j**2))**(1/3)
mass_ratio = (1+np.sqrt(1-4*eta)-2*eta)/(2*eta)
print(eta)
print(mass_ratio)

0.2116256683580524
2.2883235749605624


In [3]:
fh.ls_fm(para,[pi/4,pi/4,pi/4])

(0.014661573310839364,
 0.03301558274380271,
 0.034465210239320775,
 0.034696137802535026,
 0.11380651277246588,
 0.04424879683255985,
 0.048994053135403816,
 2.004044523805077)

In [5]:
def ls_fm(paramater,angles):
   Mtotw = 10**(paramater[0])
   M_ratiow = paramater[1]
   #rem_spin = paramater[5]


   delta_w = delta_tau = [0,0,0,0]
   para_dw = para_dtau = [0,0,0,0]

   f_ini = wf.fin(Mtotw,M_ratiow)
   f_end = wf.fout(Mtotw,M_ratiow)
   freq = np.arange(f_ini,f_end,1e-5)
   mode_delta = 1e-6
   #params_1_p = [Mtotw,M_ratiow,R_shiftw]
   #params_1_m = [Mtotw,M_ratiow,R_shiftw]   
   #对log质量的微分
   params_logmass1 = [paramater[0]+mode_delta,paramater[1],paramater[2]]
   params_logmass2 = [paramater[0]-mode_delta,paramater[1],paramater[2]]
   deriv_mass = (wf.sfa(freq,params_logmass1,angles,'lisa',delta_w,para_dtau) - wf.sfa(freq,params_logmass2,angles,'lisa',delta_w,para_dtau))/(2* mode_delta)

   #对对称质量比的微分
   params_eta1 = [paramater[0],paramater[1]+mode_delta,paramater[2]]
   params_eta2 = [paramater[0],paramater[1]-mode_delta,paramater[2]]
   deriv_eta = (wf.sfa(freq,params_eta1,angles,'lisa',delta_w,para_dtau) - wf.sfa(freq,params_eta2,angles,'lisa',delta_w,para_dtau))/(2* mode_delta)

   #omega 22 模式的微分
   delta_w22_p = delta_w[:]
   delta_w22_p[0] = mode_delta
   delta_w22_m = delta_w[:]
   delta_w22_m[0] = -mode_delta
   deriv_w22 = (wf.sfa(freq,paramater,angles,'lisa',delta_w22_p,para_dtau) - wf.sfa(freq,paramater,angles,'lisa',delta_w22_m,para_dtau))/(2* mode_delta)



   #omega 33 模式的微分
   delta_w33_p = delta_w[:]
   delta_w33_p[2] = mode_delta
   delta_w33_m = delta_w[:]
   delta_w33_m[2] = -mode_delta
   deriv_w33 = (wf.sfa(freq,paramater,angles,'lisa',delta_w33_p,para_dtau) - wf.sfa(freq,paramater,angles,'lisa',delta_w33_m,para_dtau))/(2* mode_delta)

   #omega 44 模式的微分
   delta_w44_p = delta_w[:]
   delta_w44_p[3] = mode_delta
   delta_w44_m = delta_w[:]
   delta_w44_m[3] = -mode_delta
   deriv_w44 = (wf.sfa(freq,paramater,angles,'lisa',delta_w44_p,para_dtau) - wf.sfa(freq,paramater,angles,'lisa',delta_w44_m,para_dtau))/(2* mode_delta)

   #tau 22 模式的微分
   delta_t22_p = delta_tau[:]
   delta_t22_p[0] = mode_delta
   delta_t22_m = delta_tau[:]
   delta_t22_m[0] = -mode_delta
   deriv_tau22 = (wf.sfa(freq,paramater,angles,'lisa',para_dw,delta_t22_p) - wf.sfa(freq,paramater,angles,'lisa',para_dw,delta_t22_m))/(2* mode_delta)




   #tau 33 模式的微分
   delta_t33_p = delta_tau[:]
   delta_t33_p[2] = mode_delta
   delta_t33_m = delta_tau[:]
   delta_t33_m[2] = -mode_delta
   deriv_tau33 = (wf.sfa(freq,paramater,angles,'lisa',para_dw,delta_t33_p) - wf.sfa(freq,paramater,angles,'lisa',para_dw,delta_t33_m))/(2* mode_delta)
   
   #tau 44 模式的微分
   delta_t44_p = delta_tau[:]
   delta_t44_p[3] = mode_delta
   delta_t44_m = delta_tau[:]
   delta_t44_m[3] = -mode_delta
   deriv_tau44 = (wf.sfa(freq,paramater,angles,'lisa',para_dw,delta_t44_p) - wf.sfa(freq,paramater,angles,'lisa',para_dw,delta_t44_m))/(2* mode_delta)



   diff_vec = [deriv_mass,deriv_eta,deriv_w22,deriv_w33,deriv_w44,deriv_tau22,deriv_tau33,deriv_tau44]  # Concatenate derivatives
   N_sig = 1  # Number of signals
   N_params = len(diff_vec)  # Number of parameters we care about
   K = N_sig * N_params  # Dimension of Fisher Matrix
   fish_mix = np.eye(K)
   for i in range(0,K):
      for j in range(0,K):
         fish_mix[i,j] = dp.inner_prod(diff_vec[i],diff_vec[j],wf.PSD_ls(freq),1e-5)  # Construct Fisher Matrix

   import mpmath as mp  # Import arbitrary precision matrix
   mp.dps = 4000;     # mp.dps表示精确度达到的位数 
   fish_mix_prec = mp.matrix(fish_mix)

   if mp.det(fish_mix_prec) == 0:
      raise ValueError("矩阵是奇异的，无法求逆。")

   fish_mix_inv = fish_mix_prec**-1

   #将这三行向量合成为一个矩阵
   Cov_Matrix = np.eye(K)
   for i in range(0,K):
      for j in range(0,K):
         Cov_Matrix[i,j] = float(fish_mix_inv[i,j])
         
   return np.sqrt(np.diag(Cov_Matrix))[0],np.sqrt(np.diag(Cov_Matrix))[1],np.sqrt(np.diag(Cov_Matrix))[2],np.sqrt(np.diag(Cov_Matrix))[3],\
   np.sqrt(np.diag(Cov_Matrix))[4],np.sqrt(np.diag(Cov_Matrix))[5],np.sqrt(np.diag(Cov_Matrix))[6],np.sqrt(np.diag(Cov_Matrix))[7]


In [6]:
mass = np.log10(1e6)
ls_fm([mass,2/9,1],[pi/4,pi/4,pi/4])

IndexError: list index out of range